In [ ]:
# default_exp backfill

# Pipeline for backfilling / batching

Dagster can only run one pipeline per module, and `05_pipeline.ipynb` ie `mario.py` already has one pipeline defined for continuous linear retrieval where the steps take place one after the other.



In [ ]:
#exports
import pandas as pd
import xarray as xr
import os
import glob
import dotenv
import warnings
from dagster import execute_pipeline, pipeline, solid, Field, OutputDefinition, DagsterType, Output

from IPython.display import JSON

from satip import eumetsat, reproj, io, gcp_helpers
from satip.mario import (df_metadata_to_dt_to_fp_map, 
                         reproject_datasets, 
                         save_metadata, 
                         compress_and_save_datasets, 
                         compress_export_then_delete_raw)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.30rows/s]


In [ ]:
# Filter some warnings
#exports
warnings.filterwarnings('ignore', message='divide by zero encountered in true_divide')
warnings.filterwarnings('ignore', message='invalid value encountered in sin')
warnings.filterwarnings('ignore', message='invalid value encountered in cos')
warnings.filterwarnings('ignore', message='invalid value encountered in subtract')
warnings.filterwarnings('ignore', message='You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems')

In [ ]:
missing_datasets = io.identifying_missing_datasets("2020-01-01 00:00", "2020-01-01 01:00")
JSON(missing_datasets)

Earliest 2020-01-01 00:00, latest 2020-01-01 01:00


 [████████████████████████████████████████████████████████████] 1/1 [00:08<00:08, 8.27s/it]

identify_available_datasets: found 12 results from API


<IPython.core.display.JSON object>

In [ ]:
#exports

# create pandas DataFrame type definition for Dagster
DataFrame = DagsterType(
    name="DataFrame",
    type_check_fn=lambda _, x: isinstance(x, pd.DataFrame),
)

@solid(output_defs=[OutputDefinition(name='df_new_metadata', dagster_type=DataFrame, is_required=False)])
def download_missing_eumetsat_files(context, env_vars_fp: str, data_dir: str, metadata_db_fp: str, debug_fp: str, table_id: str, project_id: str, start_date: str='', end_date: str=''):
    _ = dotenv.load_dotenv(env_vars_fp)
    dm = eumetsat.DownloadManager(os.environ.get('USER_KEY'), os.environ.get('USER_SECRET'), data_dir, metadata_db_fp, debug_fp, slack_webhook_url=os.environ.get('SLACK_WEBHOOK_URL'), slack_id=os.environ.get('SLACK_ID'))
    
    missing_datasets = io.identifying_missing_datasets(start_date, end_date)
    context.log.info(f"Missing data: {len(missing_datasets)}")
    
    df_new_metadata = dm.download_datasets(missing_datasets)

    # if df_new_metadata is None, pipeline will skip subsequent solids
    if df_new_metadata is None:
        context.log.info("*******************")
        context.log.info("Files already in zarr. Exiting.")
        context.log.info("*******************")
        return

    yield Output(df_new_metadata, 'df_new_metadata')

In [ ]:
#exports
@pipeline
def download_missing_data_pipeline():  
    # Retrieving data, reprojecting, compressing, and saving to GCP
    df_new_metadata = download_missing_eumetsat_files()
    datetime_to_filepath = df_metadata_to_dt_to_fp_map(df_new_metadata)
    ds_combined_reproj = reproject_datasets(datetime_to_filepath)
    ds_combined_compressed = compress_and_save_datasets(ds_combined_reproj)
    
    ready_to_delete = save_metadata(ds_combined_compressed, df_new_metadata)
    compress_export_then_delete_raw(ready_to_delete)

Test the configuration and execute the pipeline:

In [ ]:
#hide
run_config = {
    'solids': {
        'download_missing_eumetsat_files': {
            'inputs': {
                'env_vars_fp': "../.env",
                'data_dir': "../data/raw_bfill",
                'metadata_db_fp': "../data/EUMETSAT_metadata.db",
                'debug_fp': "../logs/EUMETSAT_download.txt",
                'table_id': "eumetsat.metadata",
                'project_id': "solar-pv-nowcasting",
                'start_date': "2019-02-01T00:00:00",
                'end_date': "2019-02-01T00:05:00"
            },
        },
        'df_metadata_to_dt_to_fp_map': {
            'inputs': {
                'data_dir': "../data/raw_bfill"
            }
        },
        'reproject_datasets': {
            'inputs': {
                'new_coords_fp': "../data/intermediate/reproj_coords_TM_4km.csv",
                'new_grid_fp': "../data/intermediate/new_grid_4km_TM.json"
            }
        },
        'compress_and_save_datasets': {
            'inputs': {
                'zarr_bucket': "solar-pv-nowcasting-data/satellite/EUMETSAT/SEVIRI_RSS/full_extent_TM_int16",
                'var_name': "stacked_eumetsat_data"
            }
        },
        'save_metadata': {
            'inputs': {
                'table_id': "eumetsat.metadata",
                'project_id': "solar-pv-nowcasting"
            },
        },
        'compress_export_then_delete_raw': {
            'inputs': {
                'data_dir': "../data/raw_bfill",
                'compressed_dir': "../data/compressed_bfill",
                'BUCKET_NAME': "solar-pv-nowcasting-data",
                'PREFIX': "satellite/EUMETSAT/SEVIRI_RSS/native/",
                'ready_to_delete': True
            },
        }
    }
}

In [ ]:
#hide
# if 'download_eumetsat_files' in run_config['solids'].keys():
#     run_config['solids']['download_missing_eumetsat_files'] = run_config['solids']['download_eumetsat_files']
#     run_config['solids'].pop('download_eumetsat_files')

execute_pipeline(download_missing_data_pipeline, run_config=run_config)

2021-03-08 09:31:09 - dagster - DEBUG - download_missing_data_pipeline - b43da3f6-03fc-4b6c-a764-774e503fbb48 - 47999 - ENGINE_EVENT - Starting initialization of resources [asset_store].
2021-03-08 09:31:09 - dagster - DEBUG - download_missing_data_pipeline - b43da3f6-03fc-4b6c-a764-774e503fbb48 - 47999 - ENGINE_EVENT - Finished initialization of resources [asset_store].
2021-03-08 09:31:09 - dagster - DEBUG - download_missing_data_pipeline - b43da3f6-03fc-4b6c-a764-774e503fbb48 - 47999 - PIPELINE_START - Started execution of pipeline "download_missing_data_pipeline".
2021-03-08 09:31:09 - dagster - DEBUG - download_missing_data_pipeline - b43da3f6-03fc-4b6c-a764-774e503fbb48 - 47999 - ENGINE_EVENT - Executing steps in process (pid: 47999)
2021-03-08 09:31:09 - dagster - DEBUG - download_missing_data_pipeline - b43da3f6-03fc-4b6c-a764-774e503fbb48 - 47999 - download_missing_eumetsat_files.compute - STEP_START - Started execution of step "download_missing_eumetsat_files.compute".
2021-0

Earliest 2019-02-01T00:00:00, latest 2019-02-01T00:05:00


 [████████████████████████████████████████████████████████████] 1/1 [00:04<00:04, 4.11s/it]

identify_available_datasets: found 1 results from API


2021-03-08 09:31:13 - dagster - INFO - system - b43da3f6-03fc-4b6c-a764-774e503fbb48 - download_missing_eumetsat_files.compute - Missing data: 1
2021-03-08 09:31:13,985 - INFO - 1 files queried, 0 found in ../data/raw_bfill, 1 to download.
2021-03-08 09:31:13,985 - INFO - 1 files queried, 0 found in ../data/raw_bfill, 1 to download.


...

1 rows written to BQ eumetsat.metadata, append=True
Found 1 native files.


2021-03-08 09:33:26 - dagster - DEBUG - system - b43da3f6-03fc-4b6c-a764-774e503fbb48 - compress_export_then_delete_raw.compute - Filesizes: Before compression = 102.210123 MB. After compression = 26.455837 MB.  Compressed file is 0.25883773762800383 x the size of the uncompressed file.
2021-03-08 09:33:26 - dagster - DEBUG - system - b43da3f6-03fc-4b6c-a764-774e503fbb48 - compress_export_then_delete_raw.compute - Moving ../data/raw_bfill/MSG3-SEVI-MSG15-0100-NA-20190201000416.813000000Z-NA.nat.bz2 to ../data/compressed_bfill/2019/02/01/00/04/MSG3-SEVI-MSG15-0100-NA-20190201000416.813000000Z-NA.nat.bz2


Moved and compressed 1 files to ../data/compressed_bfill


2021-03-08 09:34:23 - dagster - INFO - system - b43da3f6-03fc-4b6c-a764-774e503fbb48 - compress_export_then_delete_raw.compute - Removing directory ../data/raw_bfill
2021-03-08 09:34:23 - dagster - INFO - system - b43da3f6-03fc-4b6c-a764-774e503fbb48 - compress_export_then_delete_raw.compute - Removing directory ../data/compressed_bfill
2021-03-08 09:34:23 - dagster - DEBUG - download_missing_data_pipeline - b43da3f6-03fc-4b6c-a764-774e503fbb48 - 47999 - compress_export_then_delete_raw.compute - STEP_OUTPUT - Yielded output "result" of type "Any". (Type check passed).
2021-03-08 09:34:23 - dagster - DEBUG - download_missing_data_pipeline - b43da3f6-03fc-4b6c-a764-774e503fbb48 - 47999 - compress_export_then_delete_raw.compute - OBJECT_STORE_OPERATION - Stored intermediate object for output result in memory object store using pickle.
2021-03-08 09:34:23 - dagster - DEBUG - download_missing_data_pipeline - b43da3f6-03fc-4b6c-a764-774e503fbb48 - 47999 - compress_export_then_delete_raw.comp

File /Users/laurence/code/Satip/nbs/../data/compressed_bfill/2019/02/01/00/04/MSG3-SEVI-MSG15-0100-NA-20190201000416.813000000Z-NA.nat.bz2 uploaded to satellite/EUMETSAT/SEVIRI_RSS/native/2019/02/01/00/04/MSG3-SEVI-MSG15-0100-NA-20190201000416.813000000Z-NA.nat.bz2.


In [ ]:
#hide
from nbdev.export import *
notebook2script('05a_pipeline_batch.ipynb')